# 8장. 핵심 키워드 추출 (Keyword Extraction)

# 8-0 데이터 준비

## Mecab 설치 (필요시)

!sudo apt-get install g++ openjdk-7-jdk # Install Java 1.7+
!sudo apt-get install python-dev; pip install konlpy     # Python 2.x
!sudo apt-get install python3-dev; pip3 install konlpy   # Python 3.x
!sudo apt-get install curl
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

# 8-1 TF-IDF 활용 핵심키워드 추출

## 실습 1. sklearn 활용


In [1]:
import requests 
from bs4 import BeautifulSoup

def get_news_by_url(url):
    headers = {"user-agent":"Mozilla/5.0"}
    res = requests.get(url, headers=headers)
    soup = BeautifulSoup(res.text, 'html.parser')
    text = soup.select_one("div#articleBodyContents")
    
    return text.get_text()

docs = []
docs.append(get_news_by_url('https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=105&oid=018&aid=0004430108'))
docs.append(get_news_by_url('https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=001&aid=0011614790'))
docs.append(get_news_by_url('https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=014&aid=0004424362'))
docs.append(get_news_by_url('https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=119&aid=0002402191'))
docs.append(get_news_by_url('https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=030&aid=0002882728'))
len(docs)

5

### 1) 전처리

In [2]:
from konlpy.tag import Mecab
import re
mecab = Mecab()

preprocessed_docs = []

for i in docs:
    docs[docs.index(i)] = re.sub("\s*.*오류를 우회하기 위한 함수 추가\s.*\s+", "", i)
    text = re.sub("\s*.*오류를 우회하기 위한 함수 추가\s.*\s+", "", i)
    preprocessed_docs.append(" ".join([j for j,z in mecab.pos(text) 
                              if type(re.search("^[가-힣]", j)) == re.Match and z in ["NNG","VV"]]))
    
preprocessed_docs[0][:100]

'과기 정통부 장관 참석 기념행사 투입 여종 데이터 구축 민간 외부 연계 체계 개방 강화 기자 국가 차원 빅 데이터 활용 시대 산업 창출 기존 산업 변화 이르 혁신 장 센터 문 분야'

### 2) TF-IDF 계산
##### 아주 큰 말뭉치여도 보통 3만개 선에서 만듦 (oov > out of vocabulary)



In [3]:
from sklearn.feature_extraction.text import CountVectorizer

#countvectorizer() 안의 파라미터에 max_features(빈도 상한선) / max_df(df값 상한선) 등 설정 가능
count_vect = CountVectorizer(max_df=0.85)
word_count = count_vect.fit_transform(preprocessed_docs)
print(count_vect.get_feature_names()[:10])

['가공', '가동', '가운데', '가입자', '가족', '가중치', '가치', '각종', '갈리', '감소']


In [4]:
from sklearn.feature_extraction.text import TfidfTransformer

# 혹시 값이 0이되는 걸 감지해서 +1을 해주는 것 (0을 막을려는 것이다)
# use_idf는 idf를 쓸 것인지 정하는 것 (default는 trueㅠ)
tfidf_transformer = TfidfTransformer(smooth_idf=True, use_idf=True)
# fit_transform을 하면 전체에 대해 학습 끝내고 적용하는 것. 
# 우리는 하나씩 학습을 추가하고 결과를 얻어야하므로 일단 fit만!
tfidf_transformer.fit(word_count)

TfidfTransformer()

### 3) 핵심키워드 추출

In [5]:
def sort_keywords(keywords):
    return sorted(zip(keywords.col, keywords.data), key=lambda x: (x[1], x[0]), reverse=True)
    
def extract_keywords(feature_names, sorted_keywords, n=5):
    return [(feature_names[idx], score) for idx, score in sorted_keywords][:n]

In [38]:
doc = preprocessed_docs[0] # 핵심키워드 추출할 문서 조회

feature_names = count_vect.get_feature_names()
tfidf_vect = tfidf_transformer.transform(count_vect.transform([doc]))

# .tocoo()? COO형식으로 희소행렬을 표현하는 것. (행, 렬)  데이터 -> 이런 식으로 저장해줌
# .tocoo()에는 .col -> 인덱스 / .data -> 값 정보들이 들어 있음 
sorted_keywords = sort_keywords(tfidf_vect.tocoo())

    
# 사용자가 지정한 갯수만큼 키워드 추출
keywords = extract_keywords(feature_names, sorted_keywords, 5)
 
print("\n===== 원문 =====")
print(docs[0][:100])
print("\n=== 핵심키워드 ===")
for k in keywords:
    print(k)


===== 원문 =====
과기정통부, 22일 유영민 장관 등 참석해 기념행사2021년까지 1516억원 투입, 5100여종 데이터 구축민간 클라우드 통한 외부연계체계도.."개방성 강화"[이데일리 이재운 기자

=== 핵심키워드 ===
('플랫', 0.2734163036802355)
('계획', 0.23435683172591615)
('정통부', 0.19529735977159682)
('과기', 0.19529735977159682)
('혁신', 0.15756468017155242)



---


## 실습 2. gensim 활용


### 1) 전처리

In [39]:
from konlpy.tag import Mecab
mecab = Mecab()

preprocessed_docs = []
for doc in docs :
  # 명사와 동사만으로 문서 전처리
  preprocessed_docs.append(' '.join([token[0] for token in mecab.pos(doc) if token[1][0] in ['N', 'V']]))
preprocessed_docs[0][:100]

'과기 정통부 일 유영민 장관 등 참석 기념행사 년 억 원 투입 여종 데이터 구축 민간 클라우드 통한 외부 연계 체계 개방 강화 데일리 이재운 기자 국가 차원 빅 데이터 활용 시대 '

### 2) TF-IDF 계산

In [40]:
from gensim.models import TfidfModel
from gensim.corpora import Dictionary

document_ls = [doc.split() for doc in preprocessed_docs]
dct = Dictionary(document_ls) # 인덱스(key) - 단어(valuue) 인 딕셔너리 생성
corpus = [dct.doc2bow(doc) for doc in document_ls] # 각 문서에 포함된 단어를 인덱스로 변환하여 corpus 생성
tfidf = TfidfModel(corpus) # TF-IDF 산출

### 3) 핵심키워드 추출

In [41]:
def sort_keywords(tfidf):
    return sorted(tfidf, key=lambda x: (x[1], x[0]), reverse=True)

def extract_keywords(feature_names, sorted_keywords, n=5):
    return [(feature_names[idx], score) for idx, score in sorted_keywords[:n]]

In [42]:
doc = corpus[0]

sorted_keywords = sort_keywords(tfidf[doc]) # TF-IDF를 기준으로 역순 정렬

# 사용자가 지정한 갯수만큼 키워드 추출
keywords = extract_keywords(dct, sorted_keywords, 5)

print("\n=== 핵심키워드 ===")
for k in keywords:
    print(k)


=== 핵심키워드 ===
('플랫', 0.260111262735105)
('폼', 0.260111262735105)
('계획', 0.2229525109158043)
('정통부', 0.18579375909650356)
('과기', 0.18579375909650356)


In [43]:
tfidf[doc]

[(0, 0.021155348483460852),
 (1, 0.011793957648078673),
 (3, 0.023587915296157346),
 (4, 0.021155348483460852),
 (5, 0.037158751819300714),
 (6, 0.037158751819300714),
 (7, 0.037158751819300714),
 (8, 0.036063616033346915),
 (9, 0.06346604545038255),
 (10, 0.037158751819300714),
 (11, 0.042310696966921704),
 (12, 0.021155348483460852),
 (13, 0.037158751819300714),
 (14, 0.021155348483460852),
 (15, 0.037158751819300714),
 (16, 0.021155348483460852),
 (17, 0.005151945147620987),
 (18, 0.07431750363860143),
 (19, 0.10577674241730427),
 (20, 0.037158751819300714),
 (21, 0.2229525109158043),
 (22, 0.037158751819300714),
 (23, 0.010303890295241975),
 (24, 0.037158751819300714),
 (25, 0.07431750363860143),
 (26, 0.037158751819300714),
 (27, 0.011793957648078673),
 (28, 0.037158751819300714),
 (29, 0.18579375909650356),
 (30, 0.042310696966921704),
 (31, 0.037158751819300714),
 (32, 0.042310696966921704),
 (33, 0.07431750363860143),
 (34, 0.011793957648078673),
 (35, 0.037158751819300714),
 (



---



# 8-2 Textrank
https://web.eecs.umich.edu/~mihalcea/papers/mihalcea.emnlp04.pdf

<img src="https://3.bp.blogspot.com/-yp0Lr3ec5EY/XIs6znCcO_I/AAAAAAAAAPY/xtZxe_OYtH0xeuWsp4Qd4DQrunGMpVQmQCLcBGAs/s640/keyword-extraction-textrank.png" />

## 실습 1. 행렬 활용 


In [65]:
tokens = ['딸기', '바나나', '사과', '딸기', '파인애플']
nodes = ['바나나', '사과', '파인애플', '딸기']
vocab = nodes

vocab2idx = {vocab[i]:i for i in range(0, len(vocab))} #vocab을 인덱스로 변환
idx2vocab = {i:vocab[i] for i in range(0, len(vocab))} #인덱스를 vocab으로 변환
vocab2idx

{'바나나': 0, '사과': 1, '파인애플': 2, '딸기': 3}

### 3) 그래프 생성 (weighted edge 계산)

*   TextRank는 그래프 기반 모델
*   각 단어(토큰)은 그래프의 노드(vertex) 
*   weighted_edge 행렬은 노드간 가중치 정보를 담고 있음
*   weighted_edge[i][j] 는 i번째 단어와 j번째 단어의 가중치를 의미
*   weighted_edge[i][j] 가 0인 경우는 노드간 연결이 없음을 의미
*   모든 노드는 1로 초기화

In [66]:
import numpy as np
import math
vocab_len = len(vocab)

# 토큰별로 그래프 edge를 Matrix 형태로 생성
weighted_edge = np.zeros((vocab_len,vocab_len),dtype=np.float32)

# 각 토큰 노드별로 스코어 1로 초기화
score = np.ones((vocab_len),dtype=np.float32)

# coocurrence를 판단하기 위한 window 사이즈 설정 (window_size는 ngram이 아니라 span의 개념으로)
window_size = 2
covered_cooccurence = []

for window_start in range(len(tokens)-window_size+1):
    window = tokens[window_start : window_start + window_size]
    
    for i in range(window_size):
        for j in range(i+1, window_size):
            
            if (window[i] in vocab and window[j] in vocab):
                index_i = i + window_start
                index_j = j + window_start
                
                if (index_i, index_j) not in covered_cooccurence:
                    weighted_edge[vocab2idx[window[i]]][vocab2idx[window[j]]] = 1 
                    weighted_edge[vocab2idx[window[j]]][vocab2idx[window[i]]] = 1
                    covered_cooccurence.append((index_i, index_j))

                    
for i in range(vocab_len):
    row_sum = weighted_edge[i].sum()
    weighted_edge[i] = weighted_edge[i]/row_sum if row_sum > 0 else 0

weighted_edge

array([[0.        , 0.5       , 0.        , 0.5       ],
       [0.5       , 0.        , 0.        , 0.5       ],
       [0.        , 0.        , 0.        , 1.        ],
       [0.33333334, 0.33333334, 0.33333334, 0.        ]], dtype=float32)

### 4) 각 노드의 score계산
각 노드와 연결된 weighted edge의 값을 합산

In [69]:
MAX_ITERATIONS = 50
d=0.85
threshold = 0.0001 #convergence threshold

for iter in range(MAX_ITERATIONS):
    prev_score = np.copy(score)
    
    for i in range(vocab_len):
        summation = 0
        for j in range(vocab_len):
            summation += weighted_edge[j][i] * prev_score[j]
            
        score[i] = (1-d) + d*summation
        
    # np.fabs()는 float 형태의 절대값 주는 것
    if np.sum(np.fabs(prev_score - score)) <= threshold:
        print(f">>> {iter}만큼 score를 계산했어유! ^오^")
        break

score

>>> 0만큼 score를 계산했어유! ^오^


array([0.983678 , 0.9836814, 0.5656183, 1.4669032], dtype=float32)

### 5) 핵심 단어 추출

In [70]:
## np.argsort()는 argument를 sorting해서 인덱스를 뱉어줌.
## np.filp은 순서 뒤집는 것. (주어진 축에 따라서 순서를 뒤집음 / 0은 행, 1은 열)
sorted_index = np.flip(np.argsort(score),0)

n = 4

print("\n=== 핵심키워드 ===")
for i in range(n):
    print(str(idx2vocab[sorted_index[i]])+" : " + str(score[sorted_index[i]]))


=== 핵심키워드 ===
딸기 : 1.4669032
사과 : 0.9836814
바나나 : 0.983678
파인애플 : 0.5656183


In [ ]:
sorted_index[0]

3

In [204]:
import requests 
from bs4 import BeautifulSoup

def get_news_by_url(url):
    headers = {"user-agent" : "Mozilla/5.0"}
    res = requests.get(url, headers=headers)
    soup = BeautifulSoup(res.text, 'html.parser')
    text = re.sub("\s*.*오류를 우회하기 위한 함수 추가\s.*\s+","",soup.select_one("#articleBodyContents").text)
    text = re.sub(".{3} \(\w+@.+\).+\s+","",text)
    text = re.sub("[^가-힣A-z0-9\s]+"," ",text)
    text = re.sub("\s+"," ",text).strip()
    
    return text
    
doc = get_news_by_url('https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=105&oid=018&aid=0004430108')
doc

'과기정통부 22일 유영민 장관 등 참석해 기념행사2021년까지 1516억원 투입 5100여종 데이터 구축민간 클라우드 통한 외부연계체계도 개방성 강화 [이데일리 이재운 기자] 국가 차원의 빅데이터 활용 시대가 열린다 새로운 산업 창출과 기존 산업의 변화에 이르는 혁신성장 을 위한 센터가 문을 연다 10개 분야에 걸쳐 데이터 경제 의 발전을 위한 정부의 청사진을 현실로 구현하는데 앞장선다는 계획이다 22일 과학기술정보통신부는 서울 중구 대한상공회의소에서 데이터 생태계 조성과 혁신 성장의 기반 마련을 위한 빅데이터 플랫폼 및 센터 출범식 행사를 개최했다 유영민 과기정통부 장관을 비롯해 노웅래 국회 과학기술정보방송통신위원회 위원장 등 300여명이 참가했다 10개 분야 100개 센터 3년간 1516억원 투입이미지 픽사베이빅데이터는 데이터 활용을 통해 혁신성장을 이루자는 문재인 정부의 경제 성장 핵심 요소중 하나다 문재인 대통령이 직접 올 들어 데이터 활용과 이에 따른 정보보호 보안 에 대한 중요성을 강조하기도 했다 이런 맥락 속에서 빅데이터센터는 공공과 민간이 협업해 활용도 높은 양질의 데이터를 생산 구축하고 플랫폼은 이를 수집 분석 유통하는 역할을 담당한다 과기정통부는 분야별 플랫폼 10개소와 이와 연계된 기관별 센터 100개소를 구축하는데 3년간 총 1516억원을 투입할 계획이며 올해 우선 640억원 규모의 사업을 추진하고 있다 대상 분야는 금융 BC카드 환경 한국수자원공사 문화 한국문화정보원 교통 한국교통연구원 헬스케어 국립암센터 유통 소비 매일방송 통신 KT 중소기업 더존비즈온 지역경제 경기도청 산림 한국임업진흥원 등으로 현재 1차 공모를 통해 72개 빅데이터 센터를 선정했고 다음달 8일까지 2차 공모를 통해 28개를 추가 선정해 총 100개를 지원 운영할 계획이다 이를 통해 데이터 생태계를 혁신하고 기업의 경쟁력을 제고하는 역할을 수행한다 주요 활용 전략 사례를 보면 빅데이터 활용을 통해 신 시장 을 창출하는 방안을 담고 있다 금융 플랫폼의 경우 소상공인 

### 1) 토큰화 (Tokenization)

분석 텍스트 정제

In [205]:
# NNG, NNP 

from konlpy.tag import Mecab

mecab = Mecab()
tagged_text = mecab.pos(doc)

NNGP = [word for word, pos in tagged_text if pos in ["NNG", "NNP"]]

### 2) Unique한 토큰 목록 생성

그래프 생성을 위해서 Unique한 토큰 목록 생성

In [206]:
unique_nngp = list(set(NNGP))

### 어휘 사전 만들기

In [207]:
from collections import defaultdict

vocab2idx = defaultdict(lambda : len(vocab2idx))
idx2vocab = {}
for word in unique_nngp:
    vocab2idx[word]

for word, idx in vocab2idx.items():
    idx2vocab[idx] = word

### window_size 지정하고 연관도 평가

In [208]:
import numpy as np
import math

vocab_len = len(vocab2idx)

# 토큰별로 그래프 edge를 Matrix 형태로 생성
weighted_edge = np.zeros((vocab_len,vocab_len),dtype=np.float32)

# 각 토큰 노드별로 스코어 1로 초기화
score = np.ones((vocab_len),dtype=np.float32)

# coocurrence를 판단하기 위한 window 사이즈 설정
window_size = 3
covered_coocurrences = []


for window_start in range(len(NNGP)-window_size+1):
    window = NNGP[window_start : window_start + window_size]
    
    for i in range(window_size):
        for j in range(i+1, window_size):
            
            if (window[i] in unique_nngp and window[j] in unique_nngp):
                index_i = i + window_start
                index_j = j + window_start
                
                
                if (index_i, index_j) not in covered_coocurrences:
                    weighted_edge[vocab2idx[window[i]]][vocab2idx[window[j]]] = 1 
                    weighted_edge[vocab2idx[window[j]]][vocab2idx[window[i]]] = 1
                    covered_coocurrences.append((index_i, index_j))

                    
for i in range(vocab_len):
    row_sum = weighted_edge[i].sum()
    weighted_edge[i] = weighted_edge[i]/row_sum if row_sum > 0 else 0

### 연관도에 따라 점수 계산하기

In [209]:
MAX_ITERATIONS = 50
d=0.85
threshold = 0.0001 #convergence threshold


for iter in range(MAX_ITERATIONS):
    prev_score = np.copy(score)

    
    for i in range(vocab_len):
        summation = 0
        for j in range(vocab_len):
            summation += weighted_edge[j][i] * prev_score[j]
            
        score[i] = (1-d) + d*summation
        
    # np.fabs()는 float 형태의 절대값 주는 것
    if np.sum(np.fabs(prev_score - score)) <= threshold:
        print(f">>> {iter}만큼 score를 계산했습니다!")
        break
        
sorted_index = np.flip(np.argsort(score),0)

n = 5

print("\n=== 핵심키워드 ===")
for i in range(n):
    print(str(idx2vocab[sorted_index[i]])+" : " + str(score[sorted_index[i]]))

>>> 33만큼 score를 계산했습니다!

=== 핵심키워드 ===
데이터 : 10.574605
센터 : 4.1658134
활용 : 3.7810252
계획 : 3.131116
빅 : 2.5514157


---

## 실습 2. 그래프 활용 

In [234]:
import requests 
from bs4 import BeautifulSoup

def get_news_by_url(url):
    headers = {"user-agent" : "Mozilla/5.0"}
    res = requests.get(url, headers=headers)
    soup = BeautifulSoup(res.text, 'html.parser')
    text = re.sub("\s*.*오류를 우회하기 위한 함수 추가\s.*\s+","",soup.select_one("#articleBodyContents").text)
    text = re.sub(".{3} \(\w+@.+\).+\s+","",text)
    text = re.sub("[^가-힣A-z0-9\s]+"," ",text)
    text = re.sub("\s+"," ",text).strip()
    
    return text
    
doc = get_news_by_url('https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=105&oid=018&aid=0004430108')
doc

'과기정통부 22일 유영민 장관 등 참석해 기념행사2021년까지 1516억원 투입 5100여종 데이터 구축민간 클라우드 통한 외부연계체계도 개방성 강화 [이데일리 이재운 기자] 국가 차원의 빅데이터 활용 시대가 열린다 새로운 산업 창출과 기존 산업의 변화에 이르는 혁신성장 을 위한 센터가 문을 연다 10개 분야에 걸쳐 데이터 경제 의 발전을 위한 정부의 청사진을 현실로 구현하는데 앞장선다는 계획이다 22일 과학기술정보통신부는 서울 중구 대한상공회의소에서 데이터 생태계 조성과 혁신 성장의 기반 마련을 위한 빅데이터 플랫폼 및 센터 출범식 행사를 개최했다 유영민 과기정통부 장관을 비롯해 노웅래 국회 과학기술정보방송통신위원회 위원장 등 300여명이 참가했다 10개 분야 100개 센터 3년간 1516억원 투입이미지 픽사베이빅데이터는 데이터 활용을 통해 혁신성장을 이루자는 문재인 정부의 경제 성장 핵심 요소중 하나다 문재인 대통령이 직접 올 들어 데이터 활용과 이에 따른 정보보호 보안 에 대한 중요성을 강조하기도 했다 이런 맥락 속에서 빅데이터센터는 공공과 민간이 협업해 활용도 높은 양질의 데이터를 생산 구축하고 플랫폼은 이를 수집 분석 유통하는 역할을 담당한다 과기정통부는 분야별 플랫폼 10개소와 이와 연계된 기관별 센터 100개소를 구축하는데 3년간 총 1516억원을 투입할 계획이며 올해 우선 640억원 규모의 사업을 추진하고 있다 대상 분야는 금융 BC카드 환경 한국수자원공사 문화 한국문화정보원 교통 한국교통연구원 헬스케어 국립암센터 유통 소비 매일방송 통신 KT 중소기업 더존비즈온 지역경제 경기도청 산림 한국임업진흥원 등으로 현재 1차 공모를 통해 72개 빅데이터 센터를 선정했고 다음달 8일까지 2차 공모를 통해 28개를 추가 선정해 총 100개를 지원 운영할 계획이다 이를 통해 데이터 생태계를 혁신하고 기업의 경쟁력을 제고하는 역할을 수행한다 주요 활용 전략 사례를 보면 빅데이터 활용을 통해 신 시장 을 창출하는 방안을 담고 있다 금융 플랫폼의 경우 소상공인 

### 1) 토큰화 (Tokenization)

분석 텍스트 정제

In [235]:
# NNG, NNP 

from konlpy.tag import Mecab

mecab = Mecab()
tagged_text = mecab.pos(doc)

tokens = [word for word, pos in tagged_text if pos in ["NNG", "NNP"]]

In [236]:
nodes = list(set(tokens))

### 2) 그래프 생성 (weighted edge 계산)

*   TextRank는 그래프 기반 모델
*   각 단어(토큰)은 그래프의 노드(vertex) 없음을 의미
*   모든 노드는 1로 초기화

In [238]:
import numpy as np
import math
import networkx as nx

# 윈도내 동시 등장한 토큰으로 그래프를 생성
def connect(nodes, tokens) :            
    window_size = 3
    
    edges = []
    for window_start in range(len(tokens)-window_size + 1):
        window = tokens[window_start: window_start + window_size]
        
        for i in range(window_size):
            for j in range(i+1, window_size):
                if window[i] in nodes and window[j] in nodes:
                    edges.append((window[i], window[j]))
                    
    return edges

graph=nx.diamond_graph()
graph.clear() 
graph.add_nodes_from(nodes) #node 등록
graph.add_edges_from(connect(nodes, tokens)) #edge 연결    

### 3) 스코어 계산 및 핵심키워드 추출

In [242]:
scores = nx.pagerank(graph) #pagerank 계산
rank = sorted(scores.items(), key=lambda x: x[1], reverse=True) #score 역순 정렬
print("\n=== 핵심키워드 ===")
rank[:10]


=== 핵심키워드 ===


[('데이터', 0.04576257110963298),
 ('센터', 0.01802390531951102),
 ('활용', 0.01636565086611036),
 ('계획', 0.01354673917655041),
 ('빅', 0.01104723244385533),
 ('폼', 0.010925864147045706),
 ('제공', 0.010746479874439137),
 ('플랫', 0.010029375121035132),
 ('방안', 0.009931529397231329),
 ('기업', 0.009797808113842238)]

---

## 실습 3. TextRank 핵심 구 추출

### 1) 불용어를 기준으로 구 추출

In [ ]:
phrases = []
phrase = " "


print(phrases)

['과기 정통부', '일 유영민 장관 등 참석', '기념행사', '년', '억 원 투입', '여종 데이터 구축 민간 클라우드', '외부 연계 체계', '개방', '강화', '데일리 이재운 기자', '국가 차원', '빅 데이터 활용 시대', '산업 창출', '기존 산업', '변화', '혁신', '장', '센터', '문', '개 분야', '데이터 경제', '발전', '정부', '청사진', '현실', '구현', '데', '계획', '일 과학 기술 정보 통신부', '서울 중구 대한 상공 회의소', '데이터 생태', '조성', '혁신 성장', '기반 마련', '빅 데이터 플랫 폼', '센터', '출범식 행사', '개최', '유영민 과기 정통부 장관', '노웅래 국회 과학 기술 정보 방송 통신 위원회 위원장 등', '명', '참가', '개 분야', '개 센터', '년 간', '억 원 투입 이미지', '픽사 베이 빅 데이터', '데이터 활용', '혁신', '장', '문재', '정부', '경제 성장 핵심 요소 중 하나', '문재인 대통령', '데이터 활용', '이', '정보 보호', '보안', '중요', '강조', '맥락 속', '빅 데이터 센터', '공공', '민간', '협업', '활용', '양질', '데이터', '생산', '구축', '플랫 폼', '이', '수집', '분석', '유통', '역할', '담당', '과기 정통부', '분야', '플랫 폼', '개소', '이', '연계', '기관', '센터', '개소', '구축', '데', '년', '억 원', '투입', '계획', '올해', '억 원 규모', '사업', '추진', '대상 분야', '금융', '카드', '환경', '한국 수자원 공사', '문화', '한국 문화 정보원', '교통', '한국 교통 연구원', '헬 스케어', '국립암센터', '유통', '소비', '매일 방송', '통신', '중소기업', '비즈', '지역 경제', '경기도 청', '산림', '한국 임업', '흥원', '등', '차 공모', '개 빅 데이터 

In [ ]:
unique_phrases = []


print(unique_phrases)

['과기 정통부', '일 유영민 장관 등 참석', '기념행사', '년', '억 원 투입', '여종 데이터 구축 민간 클라우드', '외부 연계 체계', '개방', '강화', '데일리 이재운 기자', '국가 차원', '빅 데이터 활용 시대', '산업 창출', '기존 산업', '변화', '혁신', '장', '센터', '문', '개 분야', '데이터 경제', '발전', '정부', '청사진', '현실', '구현', '데', '계획', '일 과학 기술 정보 통신부', '서울 중구 대한 상공 회의소', '데이터 생태', '조성', '혁신 성장', '기반 마련', '빅 데이터 플랫 폼', '출범식 행사', '개최', '유영민 과기 정통부 장관', '노웅래 국회 과학 기술 정보 방송 통신 위원회 위원장 등', '명', '참가', '개 센터', '년 간', '억 원 투입 이미지', '픽사 베이 빅 데이터', '데이터 활용', '문재', '경제 성장 핵심 요소 중 하나', '문재인 대통령', '이', '정보 보호', '보안', '중요', '강조', '맥락 속', '빅 데이터 센터', '공공', '민간', '협업', '활용', '양질', '데이터', '생산', '구축', '플랫 폼', '수집', '분석', '유통', '역할', '담당', '분야', '개소', '연계', '기관', '억 원', '투입', '올해', '억 원 규모', '사업', '추진', '대상 분야', '금융', '카드', '환경', '한국 수자원 공사', '문화', '한국 문화 정보원', '교통', '한국 교통 연구원', '헬 스케어', '국립암센터', '소비', '매일 방송', '통신', '중소기업', '비즈', '지역 경제', '경기도 청', '산림', '한국 임업', '흥원', '등', '차 공모', '개 빅 데이터 센터', '선정', '다음 달', '일', '개', '추가 선정', '지원', '운영', '데이터 생태계', '기업', '경쟁력', '제고', '수행', '주요 활용 전략', '사례', '빅 데이

### 2) 각 구의 Score 계산

앞서 산출한 각 단어별 점수를 합산

In [ ]:
vocabs = dict((r[0][0],r[1]) for r in rank)
phrase_scores = []
keywords = []


Keyword: '과기 정통부', Score: 0.01124933498027082
Keyword: '일 유영민 장관 등 참석', Score: 0.013047761313401546
Keyword: '기념행사', Score: 0.0022034464344388585
Keyword: '년', Score: 0
Keyword: '억 원 투입', Score: 0.003215471038656386
Keyword: '여종 데이터 구축 민간 클라우드', Score: 0.06857882292392038
Keyword: '외부 연계 체계', Score: 0.01653296999059789
Keyword: '개방', Score: 0.007608411089591418
Keyword: '강화', Score: 0.0022598290466079364
Keyword: '데일리 이재운 기자', Score: 0.016337448780957005


### 3) 각 구를 Score로 정렬하여 핵심 구 추출

In [ ]:
sorted_index = np.flip(np.argsort(phrase_scores),0)

keywords_num = 10

print("Keywords:\n")

for i in range(0,keywords_num):
    print(str(keywords[sorted_index[i]])+", ")

Keywords:

여종 데이터 구축 민간 클라우드, 
빅 데이터 플랫 폼, 
빅 데이터 센터, 
개 빅 데이터 센터, 
빅 데이터 활용 시대, 
민간 사이 데이터 파일 형식 등, 
빅 데이터 활용, 
픽사 베이 빅 데이터, 
여종 데이터 구축, 
데이터 경제, 




---



## 실습 4. gensim 활용

In [251]:
pip freeze | grep gensim

gensim==3.6.0
Note: you may need to restart the kernel to use updated packages.


In [252]:
from gensim.summarization import keywords
keywords(doc, words=5).split('\n')

['데이터', '계획이다', '데이터를', '빅데이터', '문재인']

In [253]:
keywords?

https://github.com/lovit/KR-WordRank

## Key_sentence extraction

#### - soynlpy 만든 사람

In [262]:
# 얘도 텍스트 랭크 사용
from krwordrank.sentence import summarize_with_sentences
keywords, sents = summarize_with_sentences([doc], num_keywords=100, num_keysents=10)